In [202]:
pip install selenium


   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB 640.0 kB/s eta 0:00:16
   ---------------------------------------- 0.0/9.7 MB 640.0 kB/s eta 0:00:16
   ---------------------------------------- 0.1/9.7 MB 409.6 kB/s eta 0:00:24
   ---------------------------------------- 0.1/9.7 MB 535.8 kB/s eta 0:00:18
    --------------------------------------- 0.1/9.7 MB 522.9 kB/s eta 0:00:19
    --------------------------------------- 0.2/9.7 MB 617.3 kB/s eta 0:00:16
    --------------------------------------- 0.2/9.7 MB 655.6 kB/s eta 0:00:15
   - -------------------------------------- 0.3/9.7 MB 711.1 kB/s eta 0:00:14
   - -------------------------------------- 0.3/9.7 MB 729.6 kB/s eta 0:00:13
   - -------------------------------------- 0.4/9.7 MB 836.4 kB/s eta 0:00:12
   -- ------------------------------------- 0.5/9.7 MB 1.0 MB/s eta 0:00:09
   --

In [330]:
#importing important libraries
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [314]:
#Scraping the website
driver = webdriver.Chrome()
driver.get("https://www.pistonheads.com/buy/search")
wait = WebDriverWait(driver, 20)

while True:
    try:
        view_more_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "View more")]')))
        view_more_button.click()
        time.sleep(30)
    except Exception as e:
        #print("No more 'View More' button found:", e)
        break

page_source = driver.page_source
soup = BeautifulSoup(page_source, "html.parser")

KeyboardInterrupt: 

In [316]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, "html.parser")

In [318]:
#Creating empty lists for the table columns
Car_name = []
year = []
milage = []
engine_size = []
fuel_type = []

In [320]:
#Appending the lists with the right data
cars = soup.find_all("div", class_ = "CardWithImageAndMeta_cardContent__6SfY4 CardWithImageAndMeta_cardContentVertical__fMtPJ")
for car in cars:
    car_details = car.find("div", class_ = "CardWithImageAndMeta_contentWrapper__sWMtX CardWithImageAndMeta_contentWrapperVertical__X4grl CardOfSearchResult_contentWrapper__nfYBK")
    car_name = car_details.find("h4", class_ = "Heading_root__i6sCL Heading_noMargin__BzMFT Heading_h6__AceqR CardOfSearchResult_heading__So2W_").text.strip()
    details = car_details.find("ul", class_ = "List_root__V9_rH CardOfSearchResult_specificationList__Ba876 List_disableListStyle__dIXqN")
    all = details.find_all("li", class_ = "ListItem_root__1CjR1 CardOfSearchResult_specificationItem__4gqCy")
    if len(all) > 5:
        Car_name.append(car_name)
        year.append(all[0].text)
        milage.append(all[2].text)
        if "L" in all[3].text:
            engine_size.append(all[3].text)
        else:
            engine_size.append(0)
        if len(all) > 7:
            fuel_type.append(all[-2].text)
        elif "owne" in all[-1].text:
            fuel_type.append(all[-2].text)
        else:
            fuel_type.append(all[-1].text)

In [322]:
#converting the lists into a pandas dataframe
table = {
    "car_name": Car_name,
    "year": year,
    "milage": milage,
    "engine_size": engine_size,
    "fuel_type": fuel_type
}
data = pd.DataFrame(table)

In [324]:
data

,car_name,year,milage,engine_size,fuel_type
0,Mercedes-Benz C-Class 3.0 C43 V6 AMG (Premium ...,2020,"55,999 miles",3.0L,Petrol
1,Audi RS Q3 RS Q3 TFSI Quattro 5dr S Tronic [Co...,2020,"18,747 miles",2.5L,Petrol
2,Mazda MX-5 1.5 SKYACTIV-G Sport Nav+ Euro 6 2dr,2019,"21,847 miles",1.5L,Petrol
3,1960 TVR Grantura Lightweight,1960,"1,234 miles",1.2L,Petrol
4,PORSCHE 911 (992) GT3 TOURING,2022,"3,631 miles",4.0L,Petrol
...,...,...,...,...,...
985,Audi Q4 E-Tron,2024,"6,512 miles",0,Electric
986,Christmas cracker - ready to go,1999,"62,901 miles",4.0L,Petrol
987,Audi TT Coupe 2.0 TDI Quattro Black Edition 2d...,2013,"119,000 miles",2.0L,Diesel
988,2017 67 MASERATI LEVANTE 3.0 V6 S SUV 5DR PETR...,2017,"79,997 miles",3.0L,Petrol


In [326]:
#saving the dataset
data.to_csv("data.csv")